<a href="https://colab.research.google.com/github/oscarti2/langchaing/blob/main/Copia_de_Agents_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![picture](https://drive.google.com/uc?id=1t85VSkuEnCm-X8egDjib0GMTGZT0LM3c)

#Agentes
Preparado por Jonathan Vásquez para EvoAcademy

##Preparación
Instalación e importación de librerías.<br>
Configuración de API KEY como variable ambiente.

In [ ]:
%pip install openai langchain wikipedia

In [ ]:
import config
import os

os.environ['OPENAI_API_KEY'] = config.OPENAI_API_KEY

In [ ]:
import langchain
from langchain import Wikipedia
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.agents.react.base import DocstoreExplorer

##1. Chain
Veamos primero el caso de una llamada regular al modelo LLM. Aquí usaremos `text-davinci-003`, aunque en el paper usaron `text-davinci-002`.

In [ ]:
llm = OpenAI(temperature=0, model_name='text-davinci-003')


In [ ]:
question = "Who was the head coach of the Chile national football team when it won the Copa América for the first time?"

print(llm(question))



The head coach of the Chile national football team when it won the Copa América for the first time was Jorge Sampaoli.


In [ ]:
question = "How old was the head coach of the Chile national football team when it won the Copa América for the first time?"
print(llm(question))




The head coach of the Chile national football team when it won the Copa América for the first time was Jorge Sampaoli, who was 48 years old at the time.


Vemos que el segundo caso no es una respuesta correcta (la edad es 55). Este fenómeno es conocido como *hallucination*, y podría ser mitigado con ReAct al permitirle interactuar con el ambiente (información factual) y razonar antes de entregar una respuesta final.

##2. Agentes Basado en ReAct
Usaremos el framework [ReAct](https://react-lm.github.io/) para generar nuestro agente. Para esto usaremos el `AgentType.REACT_DOCSTORE`, que es un tipo de agente de LangChain creado para interactuar con documentos.

La interacción del agente con el documento será por medio de dos acciones:
- Search: útil para cuando se necesite buscar información. Similar a buscar un tema en el buscador de wikipedia.
- Lookup: útil para realizar búsquedas de un keyword en una documento. Similar a Cmnd+F (Alt+F) en el navegador.

Por otro lado, los documentos con los que interactuará el agente serán los contenidos en Wikipedia. Para esto usaremos `DocstoreExplorer` y el wrapper `Wikipedia()` de la API de Wikipedia.

In [ ]:
# Wikipedia : Es un wrapper de la API de Wikipedia. Retorna el resumen de una página
#            según los keywords de la query. Si no encuentra página, retorna una lista
#            de páginas similares.
#
# DocstoreExplorer: Clase que permite asistir la exploración de un documento.
#                   Tiene los métodos search y lookup.
docstore = DocstoreExplorer(Wikipedia())

# Creamos la lista de Tools que le entregaremos al agente.
# Usaremos el método docstore.search para realizar las búsquedas de doucmentos.
# y el método de docstore.lookup para buscar términos en un documento.
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description="useful for when you need to ask with search",
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description="useful for when you need to ask with lookup",
    )
]

# Inicializamos un ejecutor de agentes.
# Le entregamos el agente, la lista de tools, el modelo LLM, y el agente.
# Para rastrear las acciones, configuramos verbose=True.
react = initialize_agent(agent=AgentType.REACT_DOCSTORE,
                         tools=tools,
                         llm=llm,
                         verbose=True)


Revisemos el `prompt template` del agente.

In [ ]:
react.agent.llm_chain.prompt.template

'\n\nQuestion: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?\nThought: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.\nAction: Search[Colorado orogeny]\nObservation: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.\nThought: It does not mention the eastern sector. So I need to look up eastern sector.\nAction: Lookup[eastern sector]\nObservation: (Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.\nThought: The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.\nAction: Search[High Plains]\nObservation: High Plains refers to one of two distinct land regions\nThought: I need to instead search High Plains (United States).\nAction: Search[High Pla

```
Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action: Search[Colorado orogeny]
Observation: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.
Thought: It does not mention the eastern sector. So I need to look up eastern sector.
Action: Lookup[eastern sector]
Observation: (Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.
Thought: The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.
Action: Search[High Plains]
Observation: High Plains refers to one of two distinct land regions
Thought: I need to instead search High Plains (United States).
Action: Search[High Plains (United States)]
Observation: The High Plains are a subregion of the Great Plains. From east to west, the High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130 m).[3]
Thought: High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.
Action: Finish[1,800 to 7,000 ft]

Question: Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?
Thought: The question simplifies to "The Simpsons" character Milhouse is named after who. I only need to search Milhouse and find who it is named after.
Action: Search[Milhouse]
Observation: Milhouse Mussolini Van Houten is a recurring character in the Fox animated television series The Simpsons voiced by Pamela Hayden and created by Matt Groening.
Thought: The paragraph does not tell who Milhouse is named after, maybe I can look up "named after".
Action: Lookup[named after]
Observation: (Result 1 / 1) Milhouse was named after U.S. president Richard Nixon, whose middle name was Milhous.
Thought: Milhouse was named after U.S. president Richard Nixon, so the answer is Richard Nixon.
Action: Finish[Richard Nixon]

Question: Which documentary is about Finnish rock groups, Adam Clayton Powell or The Saimaa Gesture?
Thought: I need to search Adam Clayton Powell and The Saimaa Gesture, and find which documentary is about Finnish rock groups.
Action: Search[Adam Clayton Powell]
Observation: Could not find [Adam Clayton Powell]. Similar: [’Adam Clayton Powell III’, ’Seventh Avenue (Manhattan)’, ’Adam Clayton Powell Jr. State Office Building’, ’Isabel Washington Powell’, ’Adam Powell’, ’Adam Clayton Powell (film)’, ’Giancarlo Esposito’].
Thought: To find the documentary, I can search Adam Clayton Powell (film).
Action: Search[Adam Clayton Powell (film)]
Observation: Adam Clayton Powell is a 1989 American documentary film directed by Richard Kilberg. The film is about the rise and fall of influential African-American politician Adam Clayton Powell Jr.[3][4] It was later aired as part of the PBS series The American Experience.
Thought: Adam Clayton Powell (film) is a documentary about an African-American politician, not Finnish rock groups. So the documentary about Finnish rock groups must instead be The Saimaa Gesture.
Action: Finish[The Saimaa Gesture]

Question: What profession does Nicholas Ray and Elia Kazan have in common?
Thought: I need to search Nicholas Ray and Elia Kazan, find their professions, then find the profession they have in common.
Action: Search[Nicholas Ray]
Observation: Nicholas Ray (born Raymond Nicholas Kienzle Jr., August 7, 1911 - June 16, 1979) was an American film director, screenwriter, and actor best known for the 1955 film Rebel Without a Cause.
Thought: Professions of Nicholas Ray are director, screenwriter, and actor. I need to search Elia Kazan next and find his professions.
Action: Search[Elia Kazan]
Observation: Elia Kazan was an American film and theatre director, producer, screenwriter and actor.
Thought: Professions of Elia Kazan are director, producer, screenwriter, and actor. So profession Nicholas Ray and Elia Kazan have in common is director, screenwriter, and actor.
Action: Finish[director, screenwriter, actor]

Question: Which magazine was started first Arthur’s Magazine or First for Women?
Thought: I need to search Arthur’s Magazine and First for Women, and find which was started first.
Action: Search[Arthur’s Magazine]
Observation: Arthur’s Magazine (1844-1846) was an American literary periodical published in Philadelphia in the 19th century.
Thought: Arthur’s Magazine was started in 1844. I need to search First for Women next.
Action: Search[First for Women]
Observation: First for Women is a woman’s magazine published by Bauer Media Group in the USA.[1] The magazine was started in 1989.
Thought: First for Women was started in 1989. 1844 (Arthur’s Magazine) < 1989 (First for Women), so Arthur’s Magazine was started first.
Action: Finish[Arthur’s Magazine]

Question: Were Pavel Urysohn and Leonid Levin known for the same type of work?
Thought: I need to search Pavel Urysohn and Leonid Levin, find their types of work, then find if they are the same.
Action: Search[Pavel Urysohn]
Observation: Pavel Samuilovich Urysohn (February 3, 1898 - August 17, 1924) was a Soviet mathematician who is best known for his contributions in dimension theory.
Thought: Pavel Urysohn is a mathematician. I need to search Leonid Levin next and find its type of work.
Action: Search[Leonid Levin]
Observation: Leonid Anatolievich Levin is a Soviet-American mathematician and computer scientist.
Thought: Leonid Levin is a mathematician and computer scientist. So Pavel Urysohn and Leonid Levin have the same type of work.
Action: Finish[yes]

Question: {input}
{agent_scratchpad}
```

Tal como se plantea en el paper de `ReAct`, vemos que el `prompt template` es un conjunto de ejemplos. Esto es conocido como **few-shots**. Los ejemplos que aquí aparecen son los entregados en el paper de ReAct para la base de datos *Hotpot QA*.

Utilizando las mismas preguntas de antes, ejecutamos ahora el agente y rastreamos el paso a paso.

In [ ]:
question = "Who was the head coach of the Chile national football team when it won the Copa América for the first time?"
react.run(question)



> Entering new AgentExecutor chain...
Thought: I need to search Chile national football team, find when it won the Copa América for the first time, then find the head coach at that time.
Action: Search[Chile national football team]
Observation: The China national football team (simplified Chinese: 中国国家足球队; traditional Chinese: 中國國家足球隊; pinyin: Zhōngguó guójiā zúqiú duì, recognised as China PR by FIFA) represents the People's Republic of China in international association football and is governed by the Chinese Football Association.
China won the EAFF East Asian Cup in 2005 and 2010, was runner-up at the AFC Asian Cup in 1984 and 2004 and made its sole FIFA World Cup appearance in 2002, losing all matches without scoring a goal.
Thought: The paragraph does not mention when Chile won the Copa América for the first time. I need to search Copa América and find when Chile won it for the first time.
Action: Search[Copa América]
Observation: Could not find [Copa América]. Similar: ['Copa Am

'Jorge Sampaoli'

In [ ]:
question = "How old was the head coach of the Chile national football team when it won the Copa América for the first time?"
react.run(question)



> Entering new AgentExecutor chain...
Thought: I need to search Chile national football team, find the head coach when it won the Copa América for the first time, then find the age of the head coach.
Action: Search[Chile national football team]
Observation: The China national football team (simplified Chinese: 中国国家足球队; traditional Chinese: 中國國家足球隊; pinyin: Zhōngguó guójiā zúqiú duì, recognised as China PR by FIFA) represents the People's Republic of China in international association football and is governed by the Chinese Football Association.
China won the EAFF East Asian Cup in 2005 and 2010, was runner-up at the AFC Asian Cup in 1984 and 2004 and made its sole FIFA World Cup appearance in 2002, losing all matches without scoring a goal.
Thought: The paragraph does not mention the head coach when Chile won the Copa América for the first time. I need to search Copa América and find the head coach.
Action: Search[Copa América]
Observation: Could not find [Copa América]. Similar: ['C

'55 years old'

Vemos varios aspectos en los ejercicios:
1. Las respuestas son más concisas.
2. Evita el *hallucination*.
3. El agente es más flexible en la respuesta a resultados no esperados.

##3. Agente ReAct Zero-Shot
El problema del template anterior, es que cada llamada a la API (entre Reasoning y Act) es muy costosa, puesto que el `prompt` es muy largo. Una opción es utilizar una estrategia de zero-shot, es decir, sin ejemplos, y generar un `prompt` basado en la estrategia Plan-and-solve (ver este [paper](https://arxiv.org/pdf/2305.04091.pdf)).

LangChain nos entrega un agente llamado `AgentType.ZERO_SHOT_REACT_DESCRIPTION`, cuyo `prompt_template` está basado en zero-shot, ReAct, y agrega la descripción de todas las `tools` disponibles.

Vamos a generar nuevamente el agente que interactua con Wikipedia, pero esta vez mostraremos un ejemplo sencillo donde le entregamos una tercera herramienta que resta numeros para calcular la edad.

In [ ]:
# Esta función será utilizada para calcular la edad.
# Notar que recibe un solo argumento.
def substract(numbers)->int:
  'Compute the subtraction between two numbers'
  x = float(numbers.split(',')[0])
  y = float(numbers.split(',')[1])
  return x-y

# Creamos nuevamente la lista de tools y entregamos una descripción
# de la herramienta Substract para que el agente pueda identificar correctamente
# cuándo necesita usarla y el formato requerido.
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description="useful for when you need to ask with search",
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description="useful for when you need to ask with lookup",
    ),
    Tool.from_function(
        name='Substract',
        func=substract,
        description="useful for when you need to compute age by year and the year of birthdate separated by comma. Ex: 2023,1988",
    )
]

# Finalmente creamos el ejecutor del agente
react_zero_shot = initialize_agent(agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                                   tools=tools,
                                   llm=llm,
                                   verbose=True)


Revisemos el `prompt_template` del agente:

In [ ]:
react_zero_shot.agent.llm_chain.prompt.template

'Answer the following questions as best you can. You have access to the following tools:\n\nSearch: useful for when you need to ask with search\nLookup: useful for when you need to ask with lookup\nSubstract: useful for when you need to compute age by year and the year of birthdate separated by comma. Ex: 2023,1988\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Search, Lookup, Substract]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'

```
Answer the following questions as best you can. You have access to the following tools:

Search: useful for when you need to ask with search
Lookup: useful for when you need to ask with lookup
Substract: useful for when you need to compute age by year and the year of birthdate separated by comma. Ex: 2023,1988

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Lookup, Substract]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
```

Vemos que el `prompt` es mucho más corto. Clarametne esto afectará las interacciones. Veamos a continuación:

In [ ]:
question = "Who was the head coach of the Chile national football team when it won the Copa América for the first time?"
react_zero_shot.run(question)



> Entering new AgentExecutor chain...
 I need to find out who was the head coach of the Chile national football team when it won the Copa América for the first time.
Action: Search
Action Input: "head coach Chile national football team Copa América"
Observation: The Chile national football team (Spanish: Selección de fútbol de Chile) represents Chile in men's international football competitions and is controlled by the Federación de Fútbol de Chile which was established in 1895. The team is commonly referred to as La Roja (lit. 'The Red One'). Chile has appeared in nine World Cup tournaments and were hosts of the 1962 FIFA World Cup where they finished in third place, the highest position the country has ever achieved in the World Cup.
Chile won their first Copa América title on home soil at the 2015 Copa América, defeating Argentina in the final. They successfully defended their title in another final against Argentina won on penalties at Copa América Centenario the following year i

'Jorge Sampaoli was the head coach of the Chile national football team when it won the Copa América for the first time.'

In [ ]:
question = "How old was the head coach of the Chile national football team when it won the Copa América for the first time?"
react_zero_shot.run(question)



> Entering new AgentExecutor chain...
 I need to find out the year the Chile national football team won the Copa América for the first time and the year of birth of the head coach.
Action: Search
Action Input: "Chile national football team Copa América win"
Observation: The Chile national football team (Spanish: Selección de fútbol de Chile) represents Chile in men's international football competitions and is controlled by the Federación de Fútbol de Chile which was established in 1895. The team is commonly referred to as La Roja (lit. 'The Red One'). Chile has appeared in nine World Cup tournaments and were hosts of the 1962 FIFA World Cup where they finished in third place, the highest position the country has ever achieved in the World Cup.
Chile won their first Copa América title on home soil at the 2015 Copa América, defeating Argentina in the final. They successfully defended their title in another final against Argentina won on penalties at Copa América Centenario the followin

'The head coach of the Chile national football team when it won the Copa América for the first time was 55 years old.'

Como era de esperar, el agente lucha un poco para obtener la respuesta final. Esto podría ser mejorado agregando algunos ejemplos adicionales. Sin embargo, esto aumentaría el costo de cada llamada.

##4. Agente Pandas Dataframe
Finalmente mostraremos un ejemplo de un agente un poco más complejo. Usaremos el agente `create_pandas_dataframe_agent`, el cual crea un ejecutor con el agente `ZERO_SHOT_REACT_DESCRIPTION`, un dataframe, y con una herramienta de Python Shell. Es decir, un agente que usa python para ejecutar consultas del dataframe.

Más detalle de este agente [aquí](https://python.langchain.com/docs/integrations/toolkits/pandas).

In [ ]:
# Importamos pandas.
import pandas as pd

# Importamos la clase ChatOpenAI, pues usaremos gpt-4
from langchain.chat_models import ChatOpenAI

# Importamos el constructor del Agente Pandas Dataframe.
from langchain.agents import create_pandas_dataframe_agent

Vamos a cargar la base, creamos el modelo LLM, e instanciamos el agente:

In [ ]:
df = pd.read_csv('data-2.csv')
chat = ChatOpenAI(model_name='gpt-4',
                  temperature = 0.0)
agent_data_analyst = create_pandas_dataframe_agent(chat, df, verbose = True)

Revisemos el `prompt_template`.

In [ ]:
agent_data_analyst.agent.llm_chain.prompt.template

'\nYou are working with a pandas dataframe in Python. The name of the dataframe is `df`.\nYou should use the tools below to answer the question posed of you:\n\npython_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [python_repl_ast]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\n\nThis is the result of `print(df.head())`:\n{df_head}\n\nBegin!\nQuestion: {input}\n{agent_scratchpad}'

```
You are working with a pandas dataframe in Python. The name of the dataframe is `df`.
You should use the tools below to answer the question posed of you:

python_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [python_repl_ast]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

This is the result of `print(df.head())`:
{df_head}

Begin!
Question: {input}
{agent_scratchpad}
```

Y finalmente le haremos consultas sobre la base que le acabamos de entregar.

In [ ]:
answer = agent_data_analyst.run('¿Cuáles son los clientes top de cada país?')
print(answer)



> Entering new AgentExecutor chain...
Thought: Para responder a esta pregunta, necesitamos agrupar los datos por país y cliente, y luego sumar la cantidad de productos que cada cliente ha comprado. Luego, para cada país, seleccionamos el cliente que ha comprado la mayor cantidad de productos.

Action: python_repl_ast
Action Input: df.groupby(['Country', 'CustomerID'])['Quantity'].sum()
Observation: Country         CustomerID
Australia       12386.0         354
                12388.0        1462
                12393.0         816
                12415.0       77242
                12422.0         195
                              ...  
United Kingdom  18287.0        1586
Unspecified     12363.0         408
                12743.0         319
                14265.0         330
                16320.0         732
Name: Quantity, Length: 4380, dtype: int64
Thought:Ahora tenemos la cantidad total de productos comprados por cada cliente en cada país. Ahora necesitamos encontrar el clien

In [ ]:
answer = agent_data_analyst.run('Entrégame el producto favorito del mejor comprador del país que más compró en el mes con más compras')
print(answer)



> Entering new AgentExecutor chain...
Thought: The question is asking for the favorite product of the best buyer from the country that bought the most in the month with the most purchases. To answer this question, we need to perform several steps:

1. Identify the month with the most purchases.
2. Identify the country that bought the most in that month.
3. Identify the best buyer from that country in that month.
4. Identify the favorite product of that buyer.

Let's start with the first step: identifying the month with the most purchases. To do this, we need to convert the 'InvoiceDate' column to datetime format, extract the month, and then count the number of purchases per month.

Action: python_repl_ast
Action Input: 
```python
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['Month'] = df['InvoiceDate'].dt.month
df['Month'].value_counts().idxmax()
```
Observation: NameError: name 'pd' is not defined
Thought:I forgot to import the pandas library. I need to import it before 

---
Síguenos en nuestras redes:
- TikTok: https://www.tiktok.com/@evoacdm
- Instagram: https://www.instagram.com/evoacdm/
- LinkedIn: https://www.linkedin.com/company/evoacmd/